In [3]:
import json
import os
import shutil
import gzip
import zipfile
import urllib

from rdflib import Graph

urllib.request.urlretrieve(
    "https://data.consilium.europa.eu/data/public-voting/council-votes-on-legislative-acts.zip",
    "data/data.zip")
with zipfile.ZipFile("data/data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")


In [4]:
def process_ttl_gz(path: str):
    print(f"Processing {path}")
    with gzip.open(f"{path}.ttl.gz", 'rb') as f_in:
        with open(f"{path}.ttl", 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    if os.path.exists(path + ".json"):
        file = open(path + ".json")
        data = json.load(file)
        return data
    g = Graph()
    g.parse(path + ".ttl")
    g.serialize(path + ".json", format="json-ld")
    f = open(path + ".json")
    data = json.load(f)
    return data

In [5]:
voting_results = process_ttl_gz("data/VotingResults")
voting_results_cube = process_ttl_gz("data/VotingResultsCube")
voting_results_new = process_ttl_gz("data/VotingResultsNew")

Processing data/VotingResults
Processing data/VotingResultsCube
Processing data/VotingResultsNew


In [23]:
votes = []
num_acts = 0
votes_in_favour = []
votes_against = []
votes_abstained = []

vote_system = {}

definitions = [x for x in voting_results_cube if "http://www.w3.org/2004/02/skos/core#definition" in x]
for act in definitions:
    act_id = act["@id"]
    votes = [x for x in voting_results_cube if
             "http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/act" in x and
             "@id" in x["http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/act"][0] and
             x["http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/act"][0][
                 "@id"] == act_id]
    if len(votes) == 0:
        continue
    num_acts += 1
    current_votes_in_favour = [vote for vote in votes if
                      vote["http://data.consilium.europa.eu/data/public_voting/qb/measureproperty/vote"][0][
                          "@id"].split("/")[-1] == "votedinfavour"]
    votes_in_favour.append(len(current_votes_in_favour))

    current_votes_against = [vote for vote in votes if
                     vote["http://data.consilium.europa.eu/data/public_voting/qb/measureproperty/vote"][0][
                         "@id"].split("/")[-1] == "votedagainst"]
    votes_against.append(len(current_votes_against))
    
    current_votes_abstained = [vote for vote in votes if
                       vote["http://data.consilium.europa.eu/data/public_voting/qb/measureproperty/vote"][0][
                           "@id"].split("/")[-1] == "abstained"]
    votes_abstained.append(len(current_votes_abstained))


    print(
        f"Found {len(votes)} votes for {act_id} with {len(votes_infavour)} in favour, {len(votes_against)} against and {len(votes_abstained)} abstained ")
    print(act["http://www.w3.org/2004/02/skos/core#definition"][0]["@value"])
    vote_rule = votes[0]["http://data.consilium.europa.eu/data/public_voting/qb/dimensionproperty/votingrule"][0]["@id"].split("/")[-1]
    print(f"{vote_rule}")
    if vote_rule not in vote_system:
        vote_system[vote_rule] = 0
    else:
        vote_system[vote_rule] += 1


Found 27 votes for http://data.consilium.europa.eu/data/public_voting/consilium/act/30251 with 28 in favour, 1 against and 1 abstained 
Regulation of the European Parliament and of the Council amending Regulation (EC) No 247/2006 laying down specific measures for agriculture in the outermost regions of the Union (first reading)
qualifiedmajority
Found 28 votes for http://data.consilium.europa.eu/data/public_voting/consilium/act/332802 with 28 in favour, 2 against and 2 abstained 
Regulation of the European Parliament and of the Council establishing uniform rules and a uniform procedure for the resolution of credit institutions and certain investment firms in the framework of a Single Resolution Mechanism and a Single Resolution Fund and amending Regulation (EU) No 1093/2010 of the European Parliament and of the Council (first reading)
qualifiedmajority
Found 28 votes for http://data.consilium.europa.eu/data/public_voting/consilium/act/34281 with 28 in favour, 3 against and 3 abstained 

In [25]:
import numpy as np
print(f"Total number of acts: {num_acts}")
print(f"Total number of votes in favour: {sum(votes_in_favour)}")
print(f"Total number of votes against: {sum(votes_against)}")
print(f"Total number of votes abstained: {sum(votes_abstained)}")
print()
print(f"Min votes in favour: {min(votes_in_favour)} vs Max votes in favour: {max(votes_in_favour)}")
print(f"Mean votes in favour: {np.mean(votes_in_favour)}")
print(f"Std votes in favour: {np.std(votes_in_favour)}")
print()
print(f"Min votes against: {min(votes_against)} vs Max votes against: {max(votes_against)}")
print(f"Mean votes against: {np.mean(votes_against)}")
print(f"Std votes against: {np.std(votes_against)}")
print()
print(f"Min votes abstained: {min(votes_abstained)} vs Max votes abstained: {max(votes_abstained)}")
print(f"Mean votes abstained: {np.mean(votes_abstained)}")
print(f"Std votes abstained: {np.std(votes_abstained)}")
print()
print(f"Vote systems:\n{vote_system}")

Total number of acts: 778
Total number of votes in favour: 20806
Total number of votes against: 265
Total number of votes abstained: 278

Min votes in favour: 14 vs Max votes in favour: 104
Mean votes in favour: 26.74293059125964
Std votes in favour: 3.3948748075875654

Min votes against: 0 vs Max votes against: 8
Mean votes against: 0.34061696658097684
Std votes against: 0.9436137013077347

Min votes abstained: 0 vs Max votes abstained: 6
Mean votes abstained: 0.35732647814910024
Std votes abstained: 0.7678160185872425

Vote systems:
{'qualifiedmajority': 711, 'unanimity': 65}


In [15]:
voting_results_new

[{'@id': 'http://data.consilium.europa.eu/id/document/ST-5641-2016-INIT',
  '@type': ['http://data.consilium.europa.eu/def/codi/Document'],
  'http://data.consilium.europa.eu/def/codi/document_key': [{'@value': 'ST-5641-2016-INIT'}],
  'http://data.consilium.europa.eu/def/codi/document_number': [{'@value': '5641/2016'}]},
 {'@id': 'http://data.consilium.europa.eu/id/document/ST-12761-2011-INIT',
  '@type': ['http://data.consilium.europa.eu/def/codi/Document'],
  'http://data.consilium.europa.eu/def/codi/document_key': [{'@value': 'ST-12761-2011-INIT'}],
  'http://data.consilium.europa.eu/def/codi/document_number': [{'@value': '12761/2011'}]},
 {'@id': 'http://data.consilium.europa.eu/id/document/CM-1961-2014-INIT',
  '@type': ['http://data.consilium.europa.eu/def/codi/Document'],
  'http://data.consilium.europa.eu/def/codi/document_key': [{'@value': 'CM-1961-2014-INIT'}],
  'http://data.consilium.europa.eu/def/codi/document_number': [{'@value': '1961/2014'}]},
 {'@id': 'http://data.con